# **Semantic Image Synthesis with Spatially-Adaptive Normalization**

**Authors: Taesung Park, Ming-Yu Liu, Ting-Chun Wang, Jun-Yan ZhuL**

**Original Paper**: https://arxiv.org/pdf/1903.07291.pdf

**Official Github**: https://github.com/NVlabs/SPADE

---

**Edited By Su Hyung Choi (Key Summary & Code Practice)**

If you have any issues on this scripts, please PR to the repository below.

**[Github: @JonyChoi - Computer Vision Paper Reviews]** https://github.com/jonychoi/Computer-Vision-Paper-Reviews

Edited Jan 16 2022

---

<img src="./imgs/figure1.png" />